<a href="https://colab.research.google.com/github/essiesalari/Titanic-ML/blob/main/TitanicML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Titanic ML Project**




## Import Libraries and Dataset





iuhpuWBEKGJBN;skjdv


In [ ]:
print("essie")

essie


In [1]:
print("thank you mu dud!")

thank you mu dud!
